In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
!pip install -U efficientnet

In [ ]:
!pip install tensorflow==1.15.0
import tensorflow as tf

In [ ]:
from efficientnet.keras import EfficientNetB0


In [ ]:
import glob
import os
import shutil
import pandas as pd
import tensorflow.keras
from keras import models
from keras.models import Model
from keras import layers
import keras.backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
import numpy as np 
import statistics
import math
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import EarlyStopping
from keras.models import load_model

#Dataframe

In [ ]:
df_train = pd.read_csv('/content/gdrive/My Drive/Università/Tesi_magistrale/b_t4sa_imgs_old/df_train_pos_neg.csv')
df_val = pd.read_csv('/content/gdrive/My Drive/Università/Tesi_magistrale/b_t4sa_imgs_old/df_val_pos_neg.csv')
df_test = pd.read_csv('/content/gdrive/My Drive/Università/Tesi_magistrale/b_t4sa_imgs_old/df_test_pos_neg.csv')


In [ ]:
#CHANGE VALUE OF POS: 2-->1
df_test['Label']  = df_test['Label'].replace(to_replace=2, value=1, regex=True)
df_val['Label']   = df_val['Label'].replace(to_replace=2, value=1, regex=True)
df_train['Label'] = df_train['Label'].replace(to_replace=2, value=1, regex=True)

In [ ]:
#too much images, sample of 40000
df_train=df_train[:40000]
df_train

In [ ]:
train = pd.DataFrame()
train["img"] = df_train["IMG"]
train["label"] = df_train["Label"]


In [ ]:
val = pd.DataFrame()
val["img"] = df_val["IMG"]
val["label"] = df_val["Label"]


In [ ]:
test = pd.DataFrame()
test["img"] = df_test["IMG"]
test["label"] = df_test["Label"]


In [ ]:
frames = [val, test]
val_test = pd.concat(frames)


# Model creation


In [ ]:
model0 = EfficientNetB0(weights='imagenet',include_top=False,input_shape=(224,224,3))

model0.trainable=False


In [ ]:
W = tf.Variable(tf.random_normal([49,49], seed=0), name='weight')
b = tf.Variable(tf.random_normal([1280], seed=0), name='bias')

def all_operations(args):
    x = args[0]
    H = args[1]
    x = tf.reshape(x, [-1, 7*7,1280])
    H = tf.matmul(W, x, transpose_a=True)
    H = tf.nn.bias_add(H,b)
    H = tf.nn.softmax(H)

    x = tf.multiply(H, x)
    x = tf.reshape(x, [-1, 49*1280])
    return x

common_input = layers.Input(shape=(224, 224, 3))    
x = model0(common_input) #model0 terminate with last conv layer of EfficientNet (7,7,1280)

p = layers.Lambda(all_operations)([x, x]) #application attention mech

#p=layers.Dense(2, activation="softmax", name="fc_out")(x) #no attention mech
p=layers.Dense(2, activation="softmax", name="fc_out")(p)

model = Model(inputs=common_input, outputs=p)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

#Training

In [ ]:
train_dir="/content/gdrive/My Drive/Università/Tesi_magistrale/b_t4sa_imgs_old/"
val_dir="/content/gdrive/My Drive/Università/Tesi_magistrale/b_t4sa_imgs_old/"
test_dir="/content/gdrive/My Drive/Università/Tesi_magistrale/b_t4sa_imgs_old/"

train_datagen = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=False)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_df=pd.DataFrame()
train_df['img']=train['img']
train_df['label']=train['label'].astype(str)

#val_df=pd.DataFrame()
#val_df['img']=val['img']
#val_df['label']=val['label'].astype(str)

test_df=pd.DataFrame()
test_df['img']=val_test['img']
test_df['label']=val_test['label'].astype(str)



In [ ]:
train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=train_dir,
        x_col="img",
        y_col="label",
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')


In [ ]:
#validation_generator = test_datagen.flow_from_dataframe(
#        dataframe=val_df,
#        directory=val_dir,
#        x_col="img",
#        y_col="label",
#        target_size=(224, 224),
#        batch_size=32,
#        class_mode='categorical')

In [ ]:
test_generator = test_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory=test_dir,
        x_col="img",
        y_col="label",
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

In [ ]:
#15 epoche ma divise in più tranche, train fatto in diversi step
epoche=15
batch_size=32
history=model.fit_generator(
        train_generator,
        steps_per_epoch=(40000/batch_size),
        epochs=epoche)

In [ ]:
#model.save_weights('/content/gdrive/My Drive/Università/Tesi_magistrale/b_t4sa_imgs_old/efnet_output/efnet_giugno_no_att.h5')
#model.load_weights('/content/gdrive/My Drive/Università/Tesi_magistrale/b_t4sa_imgs_old/efnet_output/efnet_giugno_no_att.h5')


# Testing

In [ ]:
result_img = model.predict(test_generator)

In [ ]:
np.savetxt("/content/gdrive/My Drive/Università/Tesi_magistrale/b_t4sa_imgs_old/efnet_output/predict_giugno_att2.csv", result_img, delimiter=",")


In [ ]:
result_img = pd.read_csv("/content/gdrive/My Drive/Università/Tesi_magistrale/b_t4sa_imgs_old/efnet_output/predict_giugno_att2.csv", header=None)
result_text = pd.read_csv("/content/gdrive/My Drive/Università/Tesi_magistrale/b_t4sa_imgs_old/bert_prediction/predict_val_test_text.csv", header=None)


In [ ]:
preds = []
import numpy as np 
for prediction in result_img:
    #print(prediction)
    preds.append(np.argmax(prediction))
    


In [ ]:
y_test=[]
for i, row in test_df.iterrows():
  #print(row['label'])
  y_test.append(np.int64((row['label'])))



# Results

In [ ]:
#No attention
res = pd.read_csv('/content/gdrive/My Drive/Università/Tesi_magistrale/b_t4sa_imgs_old/efnet_output/predict_giugno_no_att2.csv', header=None)


In [ ]:
preds = []
import numpy as np 
for index,prediction in res.iterrows():
    #print(prediction)
    preds.append(np.argmax(prediction))

In [ ]:
y_test=[]
for i, row in test_df.iterrows():
  #print(row['label'])
  y_test.append(np.int64((row['label'])))



In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
print("Accuracy of EfNet is:",accuracy_score(y_test,preds))
print(classification_report(y_test,preds))

In [ ]:
#Attention
result_img = pd.read_csv("/content/gdrive/My Drive/Università/Tesi_magistrale/b_t4sa_imgs_old/efnet_output/predict_giugno_att2.csv", header=None)


In [ ]:
df_res=result_img.reset_index()
df_res=df_res.drop('index',axis=1)

In [ ]:
preds = []
import numpy as np 
for index,prediction in df_res.iterrows():
    #print(prediction)
    preds.append(np.argmax(prediction))

In [ ]:
y_test=[]
for i, row in test_df.iterrows():
  #print(row['label'])
  y_test.append(np.int64((row['label'])))

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
print("Accuracy of EfNet is:",accuracy_score(y_test,preds))
print(classification_report(y_test,preds))

#Extract embeddings

In [ ]:
data1=train[0:9999]
data2=train[9999:19999]
data3=train[19999:29999]
data4=train[29999:]


In [ ]:
train_df=pd.DataFrame()
train_df['img']=data1['img']
train_df['label']=data1['label'].astype(str)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=train_dir,
        x_col="img",
        y_col="label",
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')


In [ ]:
#with attention, change in model creation for no attention
m2 = Model(inputs=model.input, outputs=model.get_layer(index=2).output)
m2.summary()

In [ ]:
train_matrix = m2.predict(train_generator)

In [ ]:
np.savetxt("/content/gdrive/My Drive/Università/Tesi_magistrale/b_t4sa_imgs_old/efnet_output/to_fuse_train1_giugno_att.csv", train_matrix, delimiter=",", fmt = '%.2f')

In [ ]:
train_df=pd.DataFrame()
train_df['img']=data2['img']
train_df['label']=data2['label'].astype(str)
train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=train_dir,
        x_col="img",
        y_col="label",
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')
train_matrix = m2.predict(train_generator)
np.savetxt("/content/gdrive/My Drive/Università/Tesi_magistrale/b_t4sa_imgs_old/efnet_output/to_fuse_train2_giugno_att.csv", train_matrix, delimiter=",", fmt = '%.2f')

In [ ]:
train_df=pd.DataFrame()
train_df['img']=data3['img']
train_df['label']=data3['label'].astype(str)
train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=train_dir,
        x_col="img",
        y_col="label",
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')
train_matrix = m2.predict(train_generator)
np.savetxt("/content/gdrive/My Drive/Università/Tesi_magistrale/b_t4sa_imgs_old/efnet_output/to_fuse_train3_giugno_att.csv", train_matrix, delimiter=",", fmt = '%.2f')

In [ ]:
train_df=pd.DataFrame()
train_df['img']=data4['img']
train_df['label']=data4['label'].astype(str)
train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=train_dir,
        x_col="img",
        y_col="label",
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')
train_matrix = m2.predict(train_generator)
np.savetxt("/content/gdrive/My Drive/Università/Tesi_magistrale/b_t4sa_imgs_old/efnet_output/to_fuse_train4_giugno_no_att.csv", train_matrix, delimiter=",", fmt = '%.2f')

In [ ]:
test_generator = test_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory=test_dir,
        x_col="img",
        y_col="label",
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

In [ ]:
test_matrix = m2.predict(test_generator)

In [ ]:
np.savetxt("/content/gdrive/My Drive/Università/Tesi_magistrale/b_t4sa_imgs_old/efnet_output/to_fuse_test_giugno_att.csv", test_matrix, delimiter=",", fmt = '%.2f')
